In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as f
from PIL import Image
import os
from sklearn import metrics
import glob

In [11]:
df=pd.read_csv("data_imu_birlestirilmis.csv")

In [12]:
label_encoder=LabelEncoder()

In [13]:
df["Class Name"]=label_encoder.fit_transform(df["Class Name"])

In [14]:
dataframe=df

In [15]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51650 entries, 0 to 51649
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  51650 non-null  int64  
 1   ax          51650 non-null  float64
 2   ay          51650 non-null  float64
 3   az          51650 non-null  float64
 4   gx          51650 non-null  float64
 5   gy          51650 non-null  float64
 6   gz          51650 non-null  float64
 7   timestamp   51650 non-null  object 
 8   Class Name  51650 non-null  int32  
dtypes: float64(6), int32(1), int64(1), object(1)
memory usage: 3.3+ MB


import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class CustomDataset(Dataset):
    def __init__(self, dataframe, sequence_length=25):
        self.dataframe = dataframe
        self.features = ["ax", "ay", "az", "gx", "gy", "gz"]
        self.labels = "Class Name"
        self.sequence_length = sequence_length

        # Veriyi ölçeklendir
        self.scaler = StandardScaler()
        self.dataframe[self.features] = self.scaler.fit_transform(self.dataframe[self.features])

    def __len__(self):
        return len(self.dataframe) - self.sequence_length + 1

    def __getitem__(self, idx):
        # Özellik dizisini al
        features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
        
        # Etiketi al
        label = self.dataframe.loc[idx + self.sequence_length - 1, self.labels]
        
        # Özellikleri tensor'a dönüştür ve 3D şekil ver
        features_tensor = torch.tensor(features_sequence, dtype=torch.float32).unsqueeze(0)  # (1, sequence_length, features_count)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return features_tensor, label_tensor


In [16]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class CNNDataset(Dataset):
    
    def __init__ (self,dataframe,sequence_length=25):
        self.dataframe=dataframe
        self.features=["ax","ay","az","gx","gy","gz"]
        self.labels="Class Name"
        self.sequence_length=sequence_length

        self.scaler=StandardScaler()
        self.dataframe[self.features]=self.scaler.fit_transform(self.dataframe[self.features])
    
    def __len__(self): # kaç adet örnek ile veriyi örnekleyebileceğimiz bu şekilde hesaplıyoruz.
        return len(self.dataframe)-self.sequence_length +1
    
    def __getitem__(self,idx):

        
        features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
        
        
        label = self.dataframe.loc[idx + self.sequence_length - 1, self.labels]
        
        # Özellikleri tensöre dönüştürdükten sonra konvolüsyon için 3d ' ye çeviriyoruz
        features_tensor = torch.tensor(features_sequence, dtype=torch.float32).unsqueeze(0)  # (1, sequence_length, features_count)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return features_tensor, label_tensor

    

In [25]:
class CNNDataset(Dataset):
    
    def __init__(self, dataframe, sequence_length=25):
        self.dataframe = dataframe
        self.features = ["ax", "ay", "az", "gx", "gy", "gz"]
        self.labels = "Class Name"
        self.sequence_length = sequence_length

        self.scaler = StandardScaler()
        self.dataframe[self.features] = self.scaler.fit_transform(self.dataframe[self.features])
    
    def __len__(self):
        return len(self.dataframe) - self.sequence_length + 1
    
    def __getitem__(self, idx):
        features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
        
         
        labels_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.labels].values

        # Tüm etiketlerin aynı olup olmadığını kontrol et
        if len(set(labels_sequence)) == 1:  # Tüm etiketler aynı
            label = labels_sequence[0]
        else:  # Farklı etiketler varsa, ilgili verileri çıkar
            # Çıkarılacak indeksi güncelle
            if idx + self.sequence_length < len(self.dataframe):
                return self.__getitem__(idx + 1)
            else:
                raise IndexError("Veri setinde uygun pencere kalmadı.")

        features_tensor = torch.tensor(features_sequence, dtype=torch.float32).unsqueeze(0)  # (1, sequence_length, features_count)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return features_tensor, label_tensor


In [26]:
dataset = CNNDataset(dataframe)

 
labels = dataset.dataframe['Class Name'].values[:len(dataset)]   

 
indices = list(range(len(dataset)))

 
train_indices, test_indices = train_test_split(indices, test_size=0.2, stratify=labels)
 
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [35]:
len(train_loader)

1291

In [39]:
len(train_dataset )

41300

In [27]:
for x,y  in train_loader:
    print(x.shape)
    print(y.shape)


torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch.Size([32])
torch.Size([32, 1, 25, 6])
torch

KeyboardInterrupt: 

In [28]:
import torch
import torch.nn as nn

class CustomCNNModel(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), padding=(1, 1))
        self.pool = nn.MaxPool2d((2, 2))   

        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=(1, 1))
        
        # 1. Conv katmanı sonrası çıkış boyutu: (32, 1, 25, 6) -> (32, 32, 25, 6) -> max pooling sonrası: (32, 32, 12, 3)
        # 2. Conv katmanı sonrası çıkış boyutu: (32, 32, 12, 3) -> (32, 64, 12, 3) -> max pooling sonrası: (32, 64, 6, 1)
        # Düzleştirilmiş giriş boyutu, 64 * 6 * 1'e göre ayarlandı
        self.fc1 = nn.Linear(64 * 6 * 1, 512)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))

        # Düzleştirme işlemi
        x = x.view(-1, 64 * 6 * 1)   
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout1(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        x = torch.sigmoid(x)
        return x

 
model = CustomCNNModel()


 

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [30]:
model.to(device)

CustomCNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=384, out_features=512, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
)

In [31]:
criterion = nn.BCELoss()   # ikili sınıflnadırma için ideal
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
num_epochs=30
for epoch in range(num_epochs):
    model.train()
    for features,labels in train_loader:
        features,labels=features.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs =model(features)
        outputs = torch.squeeze(outputs)
        labels = labels.float()
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete") 

Epoch [1/30], Loss: 0.4163
Epoch [2/30], Loss: 0.3114
Epoch [3/30], Loss: 0.1827
Epoch [4/30], Loss: 0.0532
Epoch [5/30], Loss: 0.1114
Epoch [6/30], Loss: 0.0265
Epoch [7/30], Loss: 0.0006
Epoch [8/30], Loss: 0.1297
Epoch [9/30], Loss: 0.0023
Epoch [10/30], Loss: 0.0126
Epoch [11/30], Loss: 0.0460
Epoch [12/30], Loss: 0.0345
Epoch [13/30], Loss: 0.2342
Epoch [14/30], Loss: 0.0026
Epoch [15/30], Loss: 0.0049
Epoch [16/30], Loss: 0.0029
Epoch [17/30], Loss: 0.0067
Epoch [18/30], Loss: 0.0037
Epoch [19/30], Loss: 0.0012
Epoch [20/30], Loss: 0.0004
Epoch [21/30], Loss: 0.0001
Epoch [22/30], Loss: 0.0001
Epoch [23/30], Loss: 0.0000
Epoch [24/30], Loss: 0.0368
Epoch [25/30], Loss: 0.0009
Epoch [26/30], Loss: 0.0018
Epoch [27/30], Loss: 0.0027
Epoch [28/30], Loss: 0.0010
Epoch [29/30], Loss: 0.0017
Epoch [30/30], Loss: 0.1551
Training complete


In [33]:
model.eval()  # Modeli değerlendirme moduna al
test_loss = 0.0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():  
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        
        outputs = model(features)
        outputs = torch.squeeze(outputs) 
        labels = labels.float()
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Tahminleri yap
        predicted = (outputs > 0.5).float()
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)


average_test_loss = test_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

print(f'Test Loss: {average_test_loss:.4f}, Accuracy: {accuracy:.4f}')

Test Loss: 0.0026, Accuracy: 0.9995


In [34]:
# Modeli kaydet
torch.save(model.state_dict(), 'conv_model_99_23_09_2024.pth')

In [40]:
model

CustomCNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=384, out_features=512, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
)

Elbette! İşte CustomCNNModel sınıfının detaylı açıklaması:

Modelin Yapısı
Model, bir konvolüsyonel sinir ağıdır (CNN) ve aşağıdaki katmanları içerir:

Konvolüsyonel Katmanlar:

conv1:
Tip: Conv2d
Girdi: 1 kanal (örneğin, gri tonlama bir görüntü)
Çıktı: 32 kanal
Kernel Boyutu: (3, 3)
Stride: (1, 1) (Her adımda bir piksel kaydırma)
Padding: (1, 1) (Girdi boyutunu korumak için 1 piksel boşluk ekler)
Bu katman, 1 kanaldan (örneğin, gri tonlama bir görüntü) 32 kanala geçiş yaparak özellikleri çıkarır.

conv2:
Tip: Conv2d
Girdi: 32 kanal
Çıktı: 64 kanal
Kernel Boyutu: (3, 3)
Stride: (1, 1)
Padding: (1, 1)
Bu katman, bir önceki katmandan gelen 32 kanaldan daha fazla özellik çıkarır ve 64 kanal çıktısı üretir.

Havuzlama Katmanı:

pool:
Tip: MaxPool2d
Kernel Boyutu: (2, 2) (İki piksel alanında maksimum değeri alır)
Stride: (2, 2) (Her adımda iki piksel kaydırma)
Havuzlama katmanı, görüntü boyutunu azaltarak hesaplama yükünü azaltır ve modelin daha önemli özelliklere odaklanmasını sağlar.

Tam Bağlantılı Katmanlar (Fully Connected Layers):

fc1:
Tip: Linear
Girdi: 384 özellik (bu, konvolüsyon ve havuzlama işlemlerinden sonra elde edilen özellik sayısıdır)
Çıktı: 512 özellik
İlk tam bağlantılı katman, 384 girdi özelliğini alır ve 512 çıktıya dönüştürür.

dropout1:
Tip: Dropout
Oran: 0.5 (her seferinde %50 oranında birimler devre dışı bırakılır)
Dropout, aşırı uyumu (overfitting) önlemek için kullanılır. Eğitim sırasında bazı bağlantıları rastgele kapatarak modelin genelleme yeteneğini artırır.

fc2:
Tip: Linear
Girdi: 512 özellik
Çıktı: 256 özellik
İkinci tam bağlantılı katman, önceki katmandan gelen çıktıyı alarak daha az özellik sayısına indirger.

dropout2:
Tip: Dropout
Oran: 0.5
İkinci dropout katmanı da aşırı uyumu önlemeye yardımcı olur.

fc3:
Tip: Linear
Girdi: 256 özellik
Çıktı: 1 özellik
Son katman, tek bir çıkış verir. Bu, genellikle ikili sınıflandırma problemleri için kullanılır (örneğin, bir görüntünün belirli bir sınıfa ait olup olmadığını tahmin etmek).

Özet
Konvolüsyonel Katmanlar: Özellik çıkarımı için kullanılır. İki konvolüsyonel katman, girişten daha fazla özellik çıkarmak için ardışık olarak yerleştirilmiştir.
Havuzlama Katmanı: Özellik haritalarını küçültmek ve hesaplama yükünü azaltmak için kullanılır.
Tam Bağlantılı Katmanlar: Çıktıyı sınıflandırmak için kullanılır; dropout katmanları ise aşırı uyumu azaltmak için eklenmiştir.
Bu yapı, konvolüsyonel sinir ağlarının genel prensiplerini takip eder ve ikili sınıflandırma görevleri için optimize edilmiştir. Başarılar dilerim!